<a href="https://colab.research.google.com/github/tragu/Titanic-Disaster-Ludwig/blob/master/titanic_ludwig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Install Dependencies

In [0]:
!pip install https://github.com/uber/ludwig/archive/master.zip
!python -m spacy download en
!pip install -U scikit-image

Using the following commands a widget appears. Using Browse we can upload multiple or single necessary files like training data, testing data and a necessary model definition file. It will get uploaded automatically. 

In [0]:
from google.colab import files
uploaded= files.upload()

Saving model_definition.yaml.txt to model_definition.yaml.txt


The ' ! ls ' command lists the files present in the current directory.

In [0]:
!ls

gender_submission.csv  sample_data  test.csv  train.csv


Using the command ludwig train [options] we can train the data. There are lot of options and for now we use --data_csv to represent the file name of training data and --model_definition_file for specifiying model definition


In [0]:
!ludwig train \
  --data_csv train.csv \
  --model_definition_file model_definition.yaml.txt

 _         _        _      
| |_  _ __| |_ __ _(_)__ _ 
| | || / _` \ V  V / / _` |
|_|\_,_\__,_|\_/\_/|_\__, |
                     |___/ 
ludwig v0.1.0 - Train

Experiment name: experiment
Model name: run
Output path: results/experiment_run_0


ludwig_version: '0.1.0'
command: ('/usr/local/bin/ludwig train --data_csv train.csv --model_definition_file '
 'model_definition.yaml.txt')
dataset_type: 'generic'
random_seed: 42
input_data: 'train.csv'
model_definition: {   'combiner': {'type': 'concat'},
    'input_features': [   {   'name': 'Pclass',
                              'tied_weights': None,
                              'type': 'category'},
                          {   'name': 'Sex',
                              'tied_weights': None,
                              'type': 'category'},
                          {   'missing_value_strategy': 'fill_with_mean',
                              'name': 'Age',
                              'tied_weights': None,
                         

It shown that the results are stored in 'results/experiment_run_0' lets check it once


In [0]:
!ls results/experiment_run_0

description.json  model  training_statistics.json


Using predict function the test data is given and the model path 

In [0]:
!ludwig predict \
  --data_csv test.csv \
  --only_predictions\
  --model_path results/experiment_run_0/model/

 _         _        _      
| |_  _ __| |_ __ _(_)__ _ 
| | || / _` \ V  V / / _` |
|_|\_,_\__,_|\_/\_/|_\__, |
                     |___/ 
ludwig v0.1.0 - Predict

Dataset type: generic
Dataset path: test.csv
Model path: results/experiment_run_0/model/
Output path: results_0

Loading metadata from: results/experiment_run_0/model/train_set_metadata.json

╒═══════════════╕
│ LOADING MODEL │
╘═══════════════╛

  embedding_size (50) is greater than vocab_size (4). Setting embedding size to be equal to vocab_size.
  embedding_size (50) is greater than vocab_size (3). Setting embedding size to be equal to vocab_size.
  embedding_size (50) is greater than vocab_size (5). Setting embedding size to be equal to vocab_size.

╒═════════╕
│ PREDICT │
╘═════════╛

2019-03-15 13:38:50.393042: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
INFO:tensorflow:Restoring parameters from results/experiment_run

The results are stored in results_0 folder present in current folder

In [0]:
!ls results_0

Survived_predictions.csv  Survived_probabilities.csv
Survived_predictions.npy  Survived_probabilities.npy


Now we have ground truths and predicted outputs lets see them with pandas library

In [0]:
import pandas as pd
d=pd.read_csv('gender_submission.csv') #ground_truths
df=pd.read_csv('results_0/Survived_predictions.csv') #predicted results

Here the first row is made as column, we should correct it.

In [0]:
df.head()

,False
0,True
1,True
2,False
3,True
4,False


Using the following command first row [0] is inserted with 0[False] and True is changed to 1 and False is changed to 0

In [0]:
df.loc[-1] = [0]  
df.index = df.index + 1  
df = df.sort_index() 

In [0]:
d.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [0]:
df.head()

,False
0,0
1,1
2,1
3,0
4,1


Using sklearn library accuracy is calculated with inbuilt function which takes arguments [ground truth, predicted ones]

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(d['Survived'], df['False'])

0.7966507177033493